In [2]:
import boto3
ec2 = boto3.resource('ec2')
elbv2 = boto3.client('elbv2')
asg = boto3.client('autoscaling')
route53 = boto3.client('route53')

In [3]:
# Let's keep all the names in one place
load_balancer_name = 'application-load-balancer'
instances_ami_name = 'java-application-ami'
instances_security_group_name = 'instances-security-group'
balancer_security_group_name = 'balancer-security-group'
launch_configuration_name = 'asg-launch-configuration'
target_group_name = 'asg-target-group'
auto_scaling_group_name = 'auto-scaling-group'
auto_scaling_policy_name = 'auto-scaling-policy'
domain_name = 'doug-nicholson.net'
balancer_host_name = 'asg.' + domain_name

In [4]:
# Some constants
detailed_monitoring = True
minimum_instance_count = 2
desired_instance_count = 3
maximum_instance_count = 6

In [ ]:
# Choose a healthy instance at random
import random
def choose_healthy_instance():
    response = asg.describe_auto_scaling_groups(
        AutoScalingGroupNames=[
            auto_scaling_group_name
        ]
    )
    for auto_scaling_group in response['AutoScalingGroups']:
        if auto_scaling_group['AutoScalingGroupName'] == auto_scaling_group_name:
            healthy_instance_ids = [
                instance['InstanceId']
                for instance in auto_scaling_group['Instances']
                if instance['HealthStatus'] == 'Healthy'
            ]
            instance_id = random.choice(healthy_instance_ids)
            return ec2.Instance(instance_id)

In [ ]:
# Wait for a change in the status of the auto-scaling group.  If the argument
# is 0 then wait until one of the instances becomes unhealthy.  Otherwise wait
# until the number of instances specified by the argument become healthy.
import time
def wait_until_auto_scaling_group_changes_status(status_change_type, argument):
    # Check if the selected instance has become unhealthy
    def unhealthy_condition_met(auto_scaling_group, instance_id):
        instance_unhealthy = False
        for instance in auto_scaling_group['Instances']:
            if instance['InstanceId'] == instance_id:
                if instance['HealthStatus'] == 'Unhealthy':
                    instance_unhealthy = True
        return instance_unhealthy
    
    # Check if the desired number of instances are in the auto-scaling group
    def instance_count_condition_met(auto_scaling_group, desired_healthy_instance_count):
        healthy_instance_count = 0
        for instance in auto_scaling_group['Instances']:
            if instance['HealthStatus'] == 'Healthy':
                healthy_instance_count += 1
        return healthy_instance_count >= desired_healthy_instance_count
    
    # Decide which function to use
    if status_change_type == 'wait_unhealthy':
        condition_met_function = unhealthy_condition_met
    elif status_change_type == 'wait_healthy':
        condition_met_function = instance_count_condition_met
    else:
        raise('status change type is invalid')
    
    # Wait until the condition is met
    while True:
        response = asg.describe_auto_scaling_groups(
            AutoScalingGroupNames=[
                auto_scaling_group_name
            ]
        )
        for auto_scaling_group in response['AutoScalingGroups']:
            if auto_scaling_group['AutoScalingGroupName'] == auto_scaling_group_name:
                condition_met = condition_met_function(auto_scaling_group, argument)
                break
        if condition_met:
            break
        else:
            time.sleep(5)

In [ ]:
# This tests the behavior of the auto-scaling group when
# one of the instances is forcibly terminated
instance = choose_healthy_instance()
instance.terminate()

# Now wait for the auto-scaling group to notice that an instance has
# become unhealthy then wait until the instance has been restored.
wait_until_auto_scaling_group_changes_status('wait_unhealthy', instance.instance_id)
print('instance ' + instance.instance_id + ' has become unhealthy')
wait_until_auto_scaling_group_changes_status('wait_healthy', desired_instance_count)
print('there are ' + desired_instance_count + ' healthy instances again')

In [ ]:
# This tests the behavior of the auto-scaling group when
# the application on one of the instances becomes unhealthy.
import urllib
instance = choose_healthy_instance()
ip_address = instance.public_ip_address
try:
    request_url = urllib.request.urlopen('http://' + ip_address + ':4567/health/flip')
except:
    pass

# Now wait for the auto-scaling group to notice that an instance has
# become unhealthy then wait until the instance has been restored.
wait_until_auto_scaling_group_changes_status('wait_unhealthy', instance.instance_id)
print('instance ' + instance.instance_id + ' has become unhealthy')
wait_until_auto_scaling_group_changes_status('wait_healthy', desired_instance_count)
print('there are ' + desired_instance_count + ' healthy instances again')

In [5]:
# This tests the behavior of the auto-scaling group when
# put under load.  First, set the scaling policy or policies.
response = asg.put_scaling_policy(
    AutoScalingGroupName=auto_scaling_group_name,
    PolicyName=auto_scaling_policy_name,
    PolicyType='TargetTrackingScaling',
    TargetTrackingConfiguration={
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'ASGAverageCPUUtilization'
        },
        'TargetValue': 25.0,
        'DisableScaleIn': False
    },
    Enabled=True
)
# This tests the behavior of the auto-scaling group when
# the application on one of the instances becomes unhealthy.
import urllib, time
response = asg.describe_auto_scaling_groups(
    AutoScalingGroupNames=[
        auto_scaling_group_name
    ]
)
for auto_scaling_group in response['AutoScalingGroups']:
    if auto_scaling_group['AutoScalingGroupName'] == auto_scaling_group_name:
        healthy_instances = [
            ec2.Instance(instance['InstanceId'])
            for instance in auto_scaling_group['Instances']
            if instance['HealthStatus'] == 'Healthy'
        ]
        break

response = asg.describe_auto_scaling_groups(
    AutoScalingGroupNames=[
        auto_scaling_group_name
    ]
)
for auto_scaling_group in response['AutoScalingGroups']:
    if auto_scaling_group['AutoScalingGroupName'] == auto_scaling_group_name:
        healthy_instances = [
            ec2.Instance(instance['InstanceId'])
            for instance in auto_scaling_group['Instances']
            if instance['HealthStatus'] == 'Healthy'
        ]
        break
start_time = time.time()
while time.time() - start_time < 600:
    for instance in healthy_instances:
        urllib.request.urlopen('http://' + instance.public_ip_address + ':4567/cpu')
